In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from environment import run_experiment, RunParameters, RunStatistics
from baifg.model.feedback_graph import FeedbackGraph
from baifg.model.reward_model import GaussianRewardModel, RewardType
from baifg.algorithms.eps_greedy import EpsilonGreedy, EpsilonGreedyParameters
from baifg.algorithms.ucb import UCB
from baifg.algorithms.exp3g import Exp3G, Exp3GParameters
from baifg.algorithms.tas_fg import TaSFG, TaSFGParameters
from baifg.algorithms.base.graph_estimator import GraphEstimator
from baifg.algorithms.base.base_algorithm import BaseAlg
from baifg.utils.graphs import make_loopless_clique
from baifg.utils.utils import approximate_solution#, approximate_solution_new
from baifg.utils.characteristic_time import compute_characteristic_time, evaluate_characteristic_time
from itertools import product
from typing import List, NamedTuple
from tqdm import tqdm

def make_model(algo_name: BaseAlg, algo_params: NamedTuple, K: int, fg: FeedbackGraph, delta: float, informed: bool) -> BaseAlg:
    if algo_name == EpsilonGreedy:
        return EpsilonGreedy(
            GraphEstimator.optimistic_graph(K, informed=informed, known=False),
            fg.reward_model.reward_type,
            delta=delta,
            parameters=algo_params)
    elif algo_name == UCB:
        return UCB(
            GraphEstimator.optimistic_graph(K, informed=informed, known=False),
            reward_type=fg.reward_model.reward_type,
            delta=delta)
    elif algo_name == Exp3G:
        return Exp3G(
            GraphEstimator.optimistic_graph(K, informed=informed, known=False),
            reward_type=fg.reward_model.reward_type,
            delta=delta,
            parameters=algo_params
        )
    elif algo_name == TaSFG:
        return TaSFG(
            GraphEstimator.optimistic_graph(K, informed=informed, known=False),
            reward_type=fg.reward_model.reward_type,
            delta=delta, parameters=TaSFGParameters(update_frequency=2 * K)
        )
    raise Exception('Algorithm not found')



In [5]:
K=5
fg=make_loopless_clique(p=0.3, mu=np.linspace(0, 1, K))

sol = compute_characteristic_time(fg)
approx = approximate_solution(fg, normalize=True)
# approx_new = approximate_solution_new(fg.reward_model, fg.graph, normalize=True)

print(evaluate_characteristic_time(approx, fg))
# print(evaluate_characteristic_time(approx_new, fg))
print(sol.wstar)
print(approx)
# print(approx_new)

print('------')
print(sol.mstar)
print(fg.graph.G.T @ approx)
# print(fg.graph.G.T @ approx_new)
print('-----')
GG = fg.graph.G @ fg.graph.G.T

fg.reward_model.gaps
#print(fg.reward_model.mu)
# algo = make_model(algo_name=TaSFG, algo_params=TaSFGParameters(update_frequency=5, ),
#                    K=fg.K, fg=fg, delta=1e-2, informed=False)
# results = run_experiment(fg=fg, algo=algo, seed=0)
# results

281.74663856812384
[9.84336949e-01 1.56629977e-02 9.19456306e-08 0.00000000e+00
 0.00000000e+00]
[0.2358123  0.24699305 0.20830086 0.23855667 0.07033712]
------
[0.0133136  0.29530109 0.70234938 0.29765054 0.70234948]
[0.68419668 0.11368575 0.66179451 0.13284296 0.7831484 ]
-----


array([1.  , 0.75, 0.5 , 0.25, 0.25])

In [3]:
print(algo.N)
print(algo.reward.M)
print(algo.reward.mu)
print(algo.graph.G.round(2))
print(fg.graph.G)


NameError: name 'algo' is not defined